In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A


from going_modular.model.TripletFaceRecognition import EmbeddingNet, TripletNet
from going_modular.dataloader.triplet import TripletDataset
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.train_eval.triplet.train import fit
from going_modular.dataloader.triplet import CustomExrDataset
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

import os
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'epochs': 400,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

triplet_train_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=train_transform, train=True)
triplet_test_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=test_transform, train=False)

triplet_train_loader = DataLoader(
    triplet_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

triplet_test_loader = DataLoader(
    triplet_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

roc_train_dataset = CustomExrDataset(CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'])
roc_train_loader = DataLoader(
    roc_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_test_dataset = CustomExrDataset(CONFIGURATION['test_dir'], transform=test_transform, type=CONFIGURATION['type'])
roc_test_loader = DataLoader(
    roc_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

embedding_net = EmbeddingNet(len_embedding=CONFIGURATION['embedding_size']).to(device)
model = TripletNet(embedding_net).to(device)
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models')
early_min_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_loss'],
    patience=50,
    mode='min',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=10
)      

early_max_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'cosine_acc', 'euclidean_auc', 'euclidean_acc'],
    patience=50,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=20
)      

checkpoint_path = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

/home/vohoang/miniconda3/envs/torch/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2024-12-19 23:45:49.718464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 23:45:50.417423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device, 
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_max_stopping=early_max_stopping,
    early_min_stopping=early_min_stopping,
    model_checkpoint=modle_checkpoint
)

start test
start caculate metric
Success gen embedding: 5.1569664478302
Success concat all embedding to 1 tensor: 0.0005359649658203125
Success caculate score: 0.07650184631347656
Success caculate metrics: 0.1677391529083252
Success gen embedding: 0.9330594539642334
Success concat all embedding to 1 tensor: 0.0004904270172119141
Success caculate score: 0.001967906951904297
Success caculate metrics: 0.004734039306640625
Epoch 1:
	train: loss: 28.2953 | auc_cos: 0.6986acc_cos: 0.6807 | auc_eu: 0.6939 | acc_eu: 0.5560
	test: loss: 6.8756 | auc_cos: 0.7610 | acc_cos: 0.6351 | auc_eu: 0.7539 | acc_eu: 0.6575
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/triplet/albedo/models/checkpoint.pth


KeyboardInterrupt: 